<a href="https://colab.research.google.com/github/vilsonrodrigues/playing-with-falcon/blob/main/notebooks/falcon-7b-instruct-4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References: https://huggingface.co/blog/4bit-transformers-bitsandbytes

### Install dependencies

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00


### bitsandbytes configs

The 4bit integration comes with 2 different quantization types: FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the QLoRA paper

You can switch between these two dtype using bnb_4bit_quant_type from BitsAndBytesConfig. By default, the FP4 quantization is used. 

This saves more memory at no additional performance - from our empirical observations, this enables fine-tuning llama-13b model on an NVIDIA-T4 16GB with a sequence length of 1024, batch size of 1 and gradient accumulation steps of 4.

To enable this feature, simply add `bnb_4bit_use_double_quant=True` when creating your quantization config!

(text from HF colab)



We will used NF4!

In [2]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

### Load model and pipeline

In [3]:
# My version with smaller chunks on safetensors for low RAM environments
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

In [4]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
print(model_4bit)

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)


In [8]:
!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 3.2 MB/s eta 0:00:00


In [27]:
import torch
import transformers

pipeline = transformers.pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

In [10]:
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [11]:
sequences

[{'generated_text': "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron: Hello, Danny!\nDanny: You know, I've noticed that your obsession with giraffes has not changed since we last met!\nGirafatron: I love the long-legged ones!\nDanny: And here's something else I've noticed...your girafanomania may have been triggered from the fact that you never had one. Have you got a pet giraffe?\nGirafatron: No, unfortunately. I don't have enough space for one, and I'm not rich enough to buy one.\nDanny: It's a shame about that...but I have a feeling you could take care of a"}]

### Use with LangChain

In [15]:
# Some error in colab. fix with
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [16]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.0 MB/s eta 0:00:00


In [17]:
from langchain import HuggingFacePipeline

Load local LLM

In [28]:
llm = HuggingFacePipeline(pipeline=pipeline)

Define Template

In [19]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=template, 
    input_variables= ["question"]
)

Chain

In [29]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

### **Tests**

In [21]:
llm_chain("How to prepare eggs?")

{'question': 'How to prepare eggs?',
 'text': ' \nStep 1: Gather materials: a medium or large skillet, a few drops of water, and a spatula.\nStep 2: Add water to the skillet and swirl to cover the surface evenly. \nStep 3: Place the eggs in the skillet, making sure they are fully submerge in the water.\nStep 4: Let the eggs sit in the water for a few minutes. They will cook evenly as the heat from the water cooks the white and yolk.\nStep 5: Use a spatula to carefully lift the eggs out of the water and place them on a plate.\nStep 6: Let the eggs cool, if necessary, then eat. You can add salt to your eggs and enjoy a delicious omelette.'}

In [23]:
llm_chain("How to start a car?")

{'question': 'How to start a car?',
 'text': " First make sure that you're in the car, in the driver's seat, in a safe environment. Then start the engine by turning the key, pressing the pedal, or pressing your foot on the brake. The engine should then turn on, the car should vibrate, the radio should come on, the lights should turn on and off, and any other electronics should activate. Finally, you should drive off and start the engine."}

In [43]:
template2 = """Question: /n {question}. Answer: """

prompt2 = PromptTemplate(
    template=template2, 
    input_variables= ["question"]
)

In [44]:
llm_chain_2 = LLMChain(prompt=prompt2, llm=llm)

In [53]:
result_explanation = llm_chain_2("Explain antibiotics")

In [55]:
result_explanation['text']

'/n\nAntibiotics are a group of drugs that fight bacteria in our body. They work by interfering with bacterial DNA, preventing the bacteria from growing and dividing. They are used to treat a variety of bacterial infections, including strep throat, urinary tract infections, and pneumonia.'

In [ ]:
"""Author-contribution statements and acknowledgements in research papers should state clearly and specifically whether, and to what extent, the authors used AI technologies such as ChatGPT in the preparation of their manuscript and analysis. They should also indicate which LLMs were used. This will alert editors and reviewers to scrutinize manuscripts more carefully for potential biases, inaccuracies and improper source crediting. Likewise, scientific journals should be transparent about their use of LLMs, for example when selecting submitted manuscripts.
Mention the large language model based product mentioned in the paragraph above:"""

In [59]:
prompt_pt_grafos = """No ramo de análise de grafos, existe uma métrica chamada Clustering Coefficient, 
   você pode me falar como interpretar ela?"""
llm_chain_2(prompt_pt_grafos)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'No ramo de análise de grafos, existe uma métrica chamada Clustering Coefficient, \n   você pode me falar como interpretar ela?',
 'text': "/n Clustering Coefficient is a metric used in clustering algorithms to quantify the level of 'fidelity' or 'homogeneity' between groups of items. It is commonly used to identify overlapping groups and clusters in a dataset. It ranges from 1 (high) to -1 (low). The closer the value to 1, the more similar the groups and clusters are. Clustering coefficients range from 1 to 4, with 4 indicating that the dataset can be divided into four distinct groups."}

In [60]:
llm_chain_2("what is a convolution?")

{'question': 'what is a convolution?',
 'text': '/n a convolution, in the context of computer graphics, is when an image is processed in multiple dimensions, typically to blur or sharpen the image. This processing can involve convolution of pixels in the image. The term is derived from the mathematical term convolution, which is a mathematical operation that involves overlapping of two functions of the same shape (i.e., the convolution of functions of the shape). The result of the convolution operation is the resulting shape. In computer graphics, the process of convolution is performed to enhance the quality of the rendered image. This is usually achieved by using a mask to filter an image and then convolving an image with the mask. This can be done in real-time on the graphics processing unit (GPU). A convolution can involve a single dimension or multiple dimensions. If there is only one dimension, it is called 1D convolution. If two dimensions, it is called 2D convolution. If more t

In [61]:
prompt_code = """ I have to pass 2 values that are as a string

'2,3'

And turn them into a tuple

(2,3), how to do?
"""

(hit token limit)

In [62]:
llm_chain_2(prompt_code)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': " I have to pass 2 values that are as a string\n\n'2,3'\n\nAnd turn them into a tuple\n\n(2,3), how to do?\n",
 'text': '\nOne way to convert the string into a tuple using a list:\n\n- "2,3" is a string,\n- "2,3" is a pair consisting of a string and a number,\n- Therefore, you can convert the string into a list of a pair of strings, then into a tuple.\n- Here\'s an example of how you can do it:\n\n```python\nstring = "2,3"\nstring_list = list(string)\nlist_of_pairs = string_list\npairs = list(list_of_pairs)\n\n```\n\nNow `pairs` is a list containing strings like `"2,3"` and also has a pair with a string and an integer like `("2,3")`.\nTo convert this list of strings into a tuple, you can create an anonymous function to create the tuple:\n\n```python\npairs = list(",".join(s"("%d", i+1)%s%d" for d,i,i+1 in zip(string_list, string_list[i+1], int'}

In [72]:
prompt_code2 = """
/*
Write a python code to ask the user for their name and say "Hello"
*/
"""
llm_chain_2(prompt_code2)

{'question': '\n/*\nWrite a python code to ask the user for their name and say "Hello"\n*/\n',
 'text': '\nname = input( "What is your name? " )\nprint("Hello!")\nUser '}

In [95]:
llm_chain_2("How to convert a base64 file to bytes in python?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'How to convert a base64 file to bytes in python?',
 'text': "/n \nYou can convert a base64 file to bytes in python by using the 'open()' function to read the file as binary bytes, and then converting these binary bytes to strings using the 'decode()' method. Here's an example: \n\n```\nfile = open('file.b64')\nbytes = file.read()\nfile.close()\nbase64 = ''.join(bytes.decode('utf-8'))\n``` \n\nIn this example, the file `file.b64` is opened using the `open()` function and read as binary bytes using the `read()` method. The resulting binary bytes are then converted to strings using the `decode('utf-8')` method, which converts the binary bytes to a Python strings using the `utf-8` encoding. \n\nFinally, the resulting binary string is joined together using the `join()` method and a blank string to create a base64 encoded string."}

In [73]:
prompt_sql = """
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
Create a MySQL query for all students in the Computer Science Department
"""
llm_chain_2(prompt_sql)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': '\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n',
 'text': '\nThe MySQL query for this would be: \n\n```\nSELECT students.StudentId, students.StudentName \nFROM students\nWHERE departments.DepartmentId = yourDepartmentId\n``` \n\nTo select all students in the Computer Science department, you can use a similar query. The `.` character is used as the wildcard to select all columns, since only the StudentId column is required. \n'}

In [85]:
llm_chain_2("como funciona o método __call__ em python")

{'question': 'como funciona o método __call__ em python',
 'text': '/n como funciona o método __call__ em python? - /n como funciona o método __call__ em python /n como funciona o método __call__ em python\n<p>__call__ is a method in Python that takes another object as an argument and calls a method of that object. When a new instance of the object is created, the method gets called automatically. However, this behavior can be overridden with an explicit call using the __call__ method. So, to answer the question, __call__ is the method that takes another object as an argument and calls a function or method of that other object. It can be called directly without having a __call__ method defined.'}

In [98]:
llm_chain_2("show me how python's args and kwargs work")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': "show me how python's args and kwargs work",
 'text': '/n\nHere\'s an explanation and example:\n\nIn Python, you can access command-line arguments and their types using the `args` and `kwargs` arguments of a `__main__()` function.\n\n`args` is a list of arguments passed to the function, which include the names of the arguments themselves, their data types, and their order.\n\nFor example:\n\n```python\ndef hello(name, message):\n    args = [\'-n\', \'name\']\n    if len(args) > 1:\n        print(args)\n    print(args[-1])\n    message("Hello, %s!" % name.title())\n\nhello()\n```\n\nIn this example, the `arg` variable contains the arguments `-n` and `name`, which means that the `name` argument is passed with a data type of `<class\'str\'>` and is the last argument passed (after the `-n`.\n\nSimilarly, `kwargs` is used to pass keyword arguments (arguments with data types and/or names). These include arguments that are defined in the `__init__()` method of an object.\n\nFor e

In [86]:
llm_chain_2("What's latency definition?")

{'question': "What's latency definition?",
 'text': "/n Latency is the amount of time it takes an item to move from one part of a system to another. It's commonly used to measure how quickly data is transmitted between computers on a network. /n "}

In [96]:
llm_chain_2("what is Python's ABC library and what is it for?")

{'question': "what is Python's ABC library and what is it for?",
 'text': '/n The ABC library stands for Absolute Backwards Compatible. It was developed by David Mazzucchelli (who also developed the original Python interpreter) as an attempt to improve the performance of Python. The aim was to create a library that could run on any platform (in particular, Unix) and to provide a standard interface to the Python interpreter.\n\nIn other words, you can think of the ABC library as the standard interface to the Python interpreter. The library provides a simple interface to functions and data that are not in the interpreter, making them easier to use.\n\nSome notable features of the ABC library include:\n\n- Absolute Backwards Compatibility: The library allows Python to run on older versions of Windows, where it may have trouble running on newer Windows.\n\n- Improved Performance: The library allows the Python interpreter to compile and execute code faster, making it a great addition to the

In [97]:
llm_chain_2("Write me a diet, my goal is to gain lean mass and I will work out")

{'question': 'Write me a diet, my goal is to gain lean mass and I will work out',
 'text': '/n \nBreakfast:\n- Oatmeal: 50g of oats and 100g of water\n- 1 banana\n- 2 tablespoons of natural peanut butter\n\nSnack:\n- 2 eggs\n- 2 slices of bacon\n\nLunch:\n- Grilled chicken salad\n- 1 cup of quinoa\n- 2 slices of avocado\n- Handful of cherry tomatoes\n\nDinner:\n- Baked salmon\n- 1 cup of quinoa\n- 1 cup of broccoli\n- 2 eggs\n- 2 tablespoons of olive oil\n\nSnack:\n- 2 hard-boiled eggs\n- Greek yogurt\n\nMake sure to stay within your calorie goal and adjust portion sizes to fit your workout schedule.'}

In [94]:
llm_chain_2("what is the difference between Similarity embeddings and search embeddings")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'what is the difference between Similarity embeddings and search embeddings',
 'text': '/n \nSimilarity embeddings represent the relationship between two items in a high-dimensional space. Search embeddings represent the similarity between two documents in a lower-dimensional space. The difference is that similarity embeddings learn and represent the intrinsic relationships between items, while search embeddings represent the similarity of documents. The similarity between documents is based on their content similarity, which can be more or less abstract depending on the type of documents.'}

In [89]:
print(1)

1


Conversation

In [68]:
template_chat = """You are now a conversational assistant and must answer the questions: /n {history}"""

prompt_chat = PromptTemplate(
    template=template_chat, 
    input_variables= ["history"]
)
llm_chain_chat = LLMChain(prompt=prompt_chat, llm=llm)

In [64]:
prompt_conversation1 = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:
"""

In [69]:
llm_chain_chat(prompt_conversation1)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'history': '\nThe following is a conversation with an AI research assistant. The assistant tone is technical and scientific.\nHuman: Hello, who are you?\nAI: Greeting! I am an AI research assistant. How can I help you today?\nHuman: Can you tell me about the creation of blackholes?\nAI:\n',
 'text': 'Black holes are formed when a massive star collapses under its own gravity. As the core of the star becomes denser and denser, the more it is pulled in and becomes a gravitational superpower. Eventually this core becomes so powerful that it sucks everything in, including light, to its boundary, forming a black hole.'}